In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [2]:
# ── Global visualisation configuration ──────────────────────────────────────

# 1.  General Matplotlib defaults
# ── Global visualisation configuration ──────────────────────────────────────
import matplotlib as mpl
import seaborn as sns

mpl.rcParams.update({
    # ── Canvas size & resolution ───────────────────────────────────────────
    # Default figure size: 12×8 inches  →  4800×3200 px when exported at 400 dpi
    "figure.figsize": (12, 8),
    "figure.dpi": 200,       # crisp in-notebook / retina preview
    "savefig.dpi": 400,      # print-quality PNG/PDF

    # ── Fonts ──────────────────────────────────────────────────────────────
    "font.family": "sans-serif",
    "font.sans-serif": ["Roboto", "DejaVu Sans", "Arial"],
    "axes.titlesize": 24,
    # "axes.titleweight": "bold",
    "axes.labelsize": 24,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 20,

    # ── Axis & spine aesthetics ────────────────────────────────────────────
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.spines.left": True,
    "axes.spines.bottom": True,
    "axes.linewidth": 1,
    "axes.grid": True,
    "grid.color": "#E6E6E6",
    "grid.linewidth": 0.4,
    "grid.alpha": 0.8,

    # ── Colour cycle (colour-blind-safe) ───────────────────────────────────
    "axes.prop_cycle": mpl.cycler(color=sns.color_palette("Set2")),

    # ── Figure background ─────────────────────────────────────────────────
    "figure.facecolor": "white",
})

# Seaborn theme inherits the rcParams above
sns.set_theme(context="talk", style="whitegrid", palette="Set2")


# 2.  Seaborn theme (inherits Matplotlib rcParams)
sns.set_theme(
    context="talk",           # slightly larger fonts for presentations / papers
    style="whitegrid",        # grid only on y-axis (good for histograms)
    palette="Set2",           # matches the rcParams colour cycle
)

# 3.  Helper function for consistent figure export
def savefig_nice(fig, filename, *, tight=True, dpi=300, **savefig_kwargs):
    """Save figure with tight layout and correct DPI."""
    if tight:
        fig.tight_layout()
    fig.savefig(filename, dpi=dpi, bbox_inches="tight", transparent = True, **savefig_kwargs)

# 4.  Colour constants for this project (optional convenience)
COL_RAW      = sns.color_palette("Set2")[0]  # e.g. unweighted sample
COL_WEIGHTED = sns.color_palette("Set2")[1]  # weighted sample
COL_REF      = "0.35"                        # census reference (neutral grey)


In [14]:
ATLAS = "schaefer2018tian2020_400_7"
region_col = "index"
# Load important files
DATA_DIR = Path("/home/galkepler/Projects/neuroaging/data")

# Output directory for figures
OUTPUT_DIR = Path("/home/galkepler/Projects/neuroaging/figures/fig2")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Load the data
parcels = pd.read_csv(DATA_DIR / "external" /"atlases" / ATLAS / "parcels.csv", index_col = 0)
nifti = DATA_DIR / "external" / "atlases" / ATLAS / "atlas.nii.gz"
nifti_matlab = DATA_DIR / "external" / "atlases" / ATLAS / "atlas_matlab.nii"

In [4]:
metric = "gm_vol"
distribution_metric = "nanmedian"
data = pd.read_csv(DATA_DIR / "processed" / f"{metric}.csv",index_col=0).reset_index(drop=True)

/tmp/ipykernel_115033/2323632597.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_DIR / "processed" / f"{metric}.csv",index_col=0).reset_index(drop=True)


In [5]:
metric_col = "volume" if metric == "gm_vol" else distribution_metric

In [6]:
if metric == "gm_vol":
    data_global = data.drop_duplicates(subset=["subject_code"], keep="first")
else:
    data_global = data.groupby("subject_code").agg({distribution_metric: "mean","sex":"first","age_at_scan":"first"}).reset_index()

In [7]:
israel_population = pd.read_csv(DATA_DIR / "processed" / "israel_population.csv")

In [8]:
def assign_weights_by_population(
    data: pd.DataFrame,
    population_df: pd.DataFrame,
    *,
    age_col: str = "age_at_scan",
    range_start: str = "range_start",
    range_end: str = "range_end",
    total: str = "total",
) -> np.ndarray:
    """
    Post-stratification weights so the age distribution of *data*
    matches an external reference.

    - Accepts integer or float ages.
    - Handles boundary ages (include_lowest=True).
    - Leaves bins with zero sample untouched (weight=0 → later dropped).
    - Returns weights whose mean == 1 across *non-zero* weights.
    """

    # 1. Make sure population bins are well-formed
    pop = (
        population_df[[range_start, range_end, total]]
        .sort_values(range_start)
        .reset_index(drop=True)
    )
    if (pop[range_end] <= pop[range_start]).any():
        raise ValueError("Each range_end must exceed range_start.")
    if (pop[range_start].iloc[1:].values < pop[range_end].iloc[:-1].values).any():
        raise ValueError("Age bins overlap or are not strictly increasing.")

    # 2. Bin edges for pd.cut  →  [..., last_end]  (right-inclusive)
    bin_edges = pop[range_start].tolist() + [pop[range_end].iloc[-1]]

    # Population proportions (works for % or counts)
    pop_prop = pop[total] / pop[total].sum()

    # 3. Assign each participant to a bin
    sample_bins = pd.cut(
        data[age_col],
        bins=bin_edges,
        right=True,            # include right edge
        include_lowest=True,   # include first left edge
        labels=pop.index,      # integer labels 0..n-1
    )

    # 4. Sample distribution
    sample_counts = sample_bins.value_counts(sort=False)
    sample_prop = sample_counts / sample_counts.sum()

    # 5. Weight lookup  (population / sample)  — careful with zeros
    weight_lookup = pop_prop / sample_prop.replace(0, np.nan)

    # 6. Map to rows; bins with zero sample → NaN weight → drop later
    w = sample_bins.map(weight_lookup).to_numpy()

    # Assign *zero* weight (instead of 1) to rows that were NaN
    w = np.where(np.isnan(w), 0.0, w)

    # 7. Rescale so mean(weight > 0) == 1
    positive = w > 0
    w[positive] = w[positive] / w[positive].mean()

    return w


In [9]:
import numpy as np
import pandas as pd

# ─────────────────────────────────────────────────────────────────────────────
def compute_poststrat_weights(
    sample_df: pd.DataFrame,
    pop_df: pd.DataFrame,
    *,
    age_col: str = "age_at_scan",
    start_col: str = "range_start",
    end_col: str = "range_end",
    pop_total_col: str = "total",
    cap: float | None = None,
    return_bin_table: bool = False,
):
    """
    Post-stratification weights so that the age distribution of *sample_df*
    matches an external population distribution supplied in *pop_df*.

    Parameters
    ----------
    sample_df : DataFrame with an ``age_col`` column (years; int or float).
    pop_df    : DataFrame with columns
                    [start_col, end_col, pop_total_col].
                * Values in ``pop_total_col`` can be counts **or** percentages
                  (they are internally re-scaled to proportions).
                * The row order doesn’t matter; they will be sorted.
    cap       : Optional float.  If provided, weights are truncated at  
                ``cap × mean(weight)`` and then re-scaled so mean(weight)=1.
    return_bin_table : If True, also return a DataFrame summarising
                       n_sample, n_pop, and weight_factor for every bin.

    Returns
    -------
    weights   : 1-D numpy array aligned with ``sample_df.index``.
    bin_table : (optional) tidy per-bin summary (see above).
    """

    # 1 ── tidy & validate the population table --------------------------------
    pop = (
        pop_df[[start_col, end_col, pop_total_col]]
        .dropna()
        .astype({start_col: int, end_col: int, pop_total_col: float})
        .sort_values(start_col)         # your CSV is descending; fix that
        .reset_index(drop=True)
    )

    if (pop[end_col] <= pop[start_col]).any():
        raise ValueError("Each range_end must exceed range_start.")

    if (pop[start_col].iloc[1:].values < pop[end_col].iloc[:-1].values).any():
        overlap = pop.iloc[
            np.where(
                pop[start_col].iloc[1:].values < pop[end_col].iloc[:-1].values
            )[0] + 1
        ][[start_col, end_col]]
        raise ValueError(
            "Age bins overlap (rows shown below) — "
            "merge / correct them before weighting:\n"
            f"{overlap}"
        )

    # 2 ── build right-inclusive bin edges  (···| s_i  …  e_i | s_{i+1} …) ----
    edges = pop[start_col].tolist() + [pop[end_col].iloc[-1]]

    # 3 ── assign every participant to a bin -----------------------------------
    s_bins = pd.cut(
        sample_df[age_col],
        bins=edges,
        right=True,
        include_lowest=True,
        labels=pop.index,       # categorical labels 0,1,…
    )

    # 4 ── compute *sample* and *population* proportions per bin --------------
    n_sample = s_bins.value_counts(sort=False).sort_index()
    prop_sample = n_sample / n_sample.sum()

    prop_pop = pop[pop_total_col] / pop[pop_total_col].sum()

    # 5 ── weight factor = pop_prop / sample_prop ------------------------------
    weight_factor = prop_pop / prop_sample.replace(0, np.nan)   # avoid /0 → NaN

    # 6 ── map factor back to each row ----------------------------------------
    w = s_bins.map(weight_factor).astype(float).to_numpy()
    w = np.where(np.isnan(w), 0.0, w)      # rows that fell outside bins → 0

    # 7 ── optional weight truncation (winsorisation) --------------------------
    if cap is not None and cap > 0:
        mean_pos = w[w > 0].mean()
        w = np.clip(w, 0, cap * mean_pos)

    # 8 ── re-scale so mean(weight > 0) == 1 -----------------------------------
    positive = w > 0
    w[positive] = w[positive] / w[positive].mean()

    # 9 ── return --------------------------------------------------------------
    if return_bin_table:
        bin_tbl = pd.DataFrame(
            {
                "n_sample": n_sample,
                "n_pop": prop_pop * n_sample.sum(),   # rescaled to sample size
                "prop_sample": prop_sample,
                "prop_pop": prop_pop,
                "weight_factor": weight_factor,
            }
        )
        return w, bin_tbl

    return w


In [10]:
# assign weights
data["weight"],_ = compute_poststrat_weights(
    data,
    israel_population,
    age_col="age_at_scan",
    cap=None,  # try None first; if still spiky, use 4 or 5
    return_bin_table=True,
)

# change sex into numeric
data["sex"] = data["sex"].map({"M": 0, "F": 1})

In [11]:
from statsmodels.formula.api import ols, wls

def regional_models(region_data:pd.DataFrame, metric_col:str, model:str, covariates:list):
    """
    Fit a linear model to the data for each region.
    """
    # Fit the model
    if model == "ols":
        model = ols(f"{metric_col} ~ {' + '.join(covariates)}", data=region_data)
    elif model == "wls":
        model = wls(f"{metric_col} ~ {' + '.join(covariates)}", data=region_data, weights=region_data["weight"])
    else:
        raise ValueError("Model must be either 'ols' or 'wls'.")

    # Fit the model
    results = model.fit()

    # Return the results
    return results

In [12]:
from tqdm import tqdm
# Set up dataframes to store results
lin_unw, lin_w, quad_unw, quad_w = parcels.copy(), parcels.copy(), parcels.copy(), parcels.copy()

covariates = ["sex"]
if metric == "gm_vol":
    covariates += ["tiv"]

for i, row in tqdm(parcels.iterrows(), total=parcels.shape[0], desc="Fitting models"):
    region_data = data[data[region_col] == row[region_col]].copy()
    region_data.rename(columns={metric_col: "value"}, inplace=True)
    for results_df, model, is_quadratic in zip(
        [lin_unw, lin_w, quad_unw, quad_w],
        ["ols", "wls", "ols", "wls"],
        [False, False, True, True],
    ):
        # Fit the model
        if is_quadratic:
            region_data["age_squared"] = region_data["age_at_scan"] ** 2
            results = regional_models(region_data, "value", model, covariates + ["age_squared"])
        else:
            results = regional_models(region_data, "value", model, covariates)

        # Store the results
        results_df.loc[i, "model"] = results
        results_df.loc[i, "pvalue"] = results.f_pvalue
        results_df.loc[i, "r2"] = results.rsquared
        results_df.loc[i, "r2_adj"] = results.rsquared_adj
        results_df.loc[i, "n"] = len(region_data)

Fitting models: 100%|██████████| 454/454 [00:08<00:00, 55.15it/s]


In [13]:
from statsmodels.stats.anova import anova_lm

unw_compare = parcels.copy()
w_compare = parcels.copy()

# do f test to compare between quadratic and linear models
for i, row in tqdm(parcels.iterrows(), total=parcels.shape[0], desc="Fitting models"):
    for results_lin, results_quad, results_df in zip(
        [lin_unw, lin_w],
        [quad_unw, quad_w],
        [unw_compare, w_compare],
    ):
        # collect models
        model1 = results_lin.loc[i, "model"]
        model2 = results_quad.loc[i, "model"]
        f, p, _ = model2.compare_f_test(model1)
        results_df.loc[i, "f"] = f
        results_df.loc[i, "pvalue"] = p
        results_df.loc[i, "n"] = model1.nobs
        # do f test to compare between quadratic and linear models
    # model1 = lin_unw.loc[i, "model"]
    # model2 = quad_unw.loc[i, "model"]
    # f, p, _ = model2.compare_f_test(model1)
    # unw_compare.loc[i, "f"] = f
    # unw_compare.loc[i, "pvalue"] = p
    # unw_compare.loc[i, "n"] = model1.nobs
    # weighted

Fitting models: 100%|██████████| 454/454 [00:00<00:00, 1531.14it/s]


In [15]:
# correct everything for multiple comparisons
from statsmodels.stats.multitest import multipletests

for results_df, save_name in zip([lin_unw, lin_w, quad_unw, quad_w, unw_compare, w_compare], ["lin_unw", "lin_w", "quad_unw", "quad_w", "unw_compare", "w_compare"]):
    results_df["p_corrected"] = multipletests(results_df["pvalue"], method="fdr_bh")[1]
    results_df.to_csv(OUTPUT_DIR / f"{save_name}_{metric}.csv", index=False)

# lin_unw["p_corrected"] = multipletests(lin_unw["pvalue"], method="fdr_bh")[1]
# lin_unw.to_csv(OUTPUT_DIR / f"lin_unw_{metric}.csv", index=False)

# lin_w["p_corrected"] = multipletests(lin_w["pvalue"], method="fdr_bh")[1]
# lin_w.to_csv(OUTPUT_DIR / f"lin_w_{metric}.csv", index=False)

# quad_unw["p_corrected"] = multipletests(quad_unw["pvalue"], method="fdr_bh")[1]
# quad_unw.to_csv(OUTPUT_DIR / f"quad_unw_{metric}.csv", index=False)

# quad_w["p_corrected"] = multipletests(quad_w["pvalue"], method="fdr_bh")[1]
# unw_compare["p_corrected"] = multipletests(unw_compare["pvalue"], method="fdr_bh")[1]
# w_compare["p_corrected"] = multipletests(w_compare["pvalue"], method="fdr_bh")[1]


In [55]:
import numpy as np
import pandas as pd
import nibabel as nb
from nilearn import surface, datasets
from surfplot import Plot
from matplotlib.colors import TwoSlopeNorm  # nice diverging colours

atlas_img = nib.load(nifti)

# ---------------------------------------------------------------------
# 2.  FETCH A STANDARD SURFACE  (fsaverage5 = 10k vertices per hemi)
# ---------------------------------------------------------------------
fsavg = datasets.fetch_surf_fsaverage("fsaverage5")  # ~5 MB download

# ---------------------------------------------------------------------
# 3.  SAMPLE ATLAS VOXELS → SURFACE  (nearest-neighbour so labels stay int)
# ---------------------------------------------------------------------
labels_lh = surface.vol_to_surf(atlas_img, fsavg["pial_left"], interpolation="nearest").astype(int)
labels_rh = surface.vol_to_surf(atlas_img, fsavg["pial_right"], interpolation="nearest").astype(
    int
)

In [58]:
# ---------------------------------------------------------------------
# 4.  MAP REGION IDs → METRIC VALUES
#     vertices with label 0 (background) → NaN so they render transparent
# ---------------------------------------------------------------------
p_threshold = 0.05

value_map = {}

vis_df = lin_unw
for i, row in vis_df.iterrows():
    value_map[row[region_col]] = row["r2_adj"] if row["p_corrected"] < p_threshold else np.nan

vec = np.vectorize(lambda x: value_map.get(x, np.nan))
data_lh = vec(labels_lh)
data_rh = vec(labels_rh)

In [63]:
# ---------------------------------------------------------------------
# 5.  BUILD THE PLOT
# ---------------------------------------------------------------------
p = Plot(
    fsavg["infl_left"],
    fsavg["infl_right"],  # inflated surfaces
    # views="latmed",  # L-lat, L-med, R-lat, R-med
    size=(1800, 800),  # px; change as needed
)

# ---- main data layer -------------------------------------------------
# If your metric is centred on 0 (e.g., t-values) use TwoSlopeNorm for symmetry
# norm = TwoSlopeNorm(
#     vmin=np.nanmin([data_lh, data_rh]), ,vmax=np.nanmax([data_lh, data_rh])
# )

p.add_layer(
    {"left": data_lh, "right": data_rh},
    cmap="coolwarm",
    # color_range=
    cbar_label="My metric",
)

# ---- outline layer ---------------------------------------------------
# Re-use the label arrays; surfplot draws borders when as_outline=True
p.add_layer({"left": labels_lh, "right": labels_rh}, cmap="gray", as_outline=True, cbar=False)

fig = p.build(cbar_kws=dict(location="bottom", decimals=2))

: 